In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# in order to import functions in src folder
import sys
sys.path.append('C:\\Users\\TinaM\\Desktop\\TMB_File\\UTS_JUN_2023\\AdvDSL_36114\\AT2\\ADSI_AT2')
print(sys.path)

['c:\\Users\\TinaM\\Desktop\\TMB_File\\UTS_JUN_2023\\AdvDSL_36114\\AT2\\ADSI_AT2\\notebooks', 'C:\\Users\\TinaM\\Desktop\\TMB_File\\UTS_JUN_2023\\AdvDSL_36114\\AT2\\ADSI_AT2\\src', 'C:\\Users\\TinaM\\AppData\\Local\\Programs\\Python\\Python38\\python38.zip', 'C:\\Users\\TinaM\\AppData\\Local\\Programs\\Python\\Python38\\DLLs', 'C:\\Users\\TinaM\\AppData\\Local\\Programs\\Python\\Python38\\lib', 'C:\\Users\\TinaM\\AppData\\Local\\Programs\\Python\\Python38', 'c:\\Users\\TinaM\\.virtualenvs\\ADSI_AT2-VD9S5n3N', '', 'c:\\Users\\TinaM\\.virtualenvs\\ADSI_AT2-VD9S5n3N\\lib\\site-packages', 'c:\\Users\\TinaM\\.virtualenvs\\ADSI_AT2-VD9S5n3N\\lib\\site-packages\\win32', 'c:\\Users\\TinaM\\.virtualenvs\\ADSI_AT2-VD9S5n3N\\lib\\site-packages\\win32\\lib', 'c:\\Users\\TinaM\\.virtualenvs\\ADSI_AT2-VD9S5n3N\\lib\\site-packages\\Pythonwin', 'C:\\Users\\TinaM\\Desktop\\TMB_File\\UTS_JUN_2023\\AdvDSL_36114\\AT2\\ADSI_AT2']


In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
rawfile_path = "../data/raw/beer_reviews.csv"
datadict_path = "../references/Data_Dict.csv"
beer_style_list_path = "../data/processed/beer_style"
brewery_name_list_path = "../data/processed/brewery_name"
knn_imputer_path = "../models/knn_imputer_numeric.joblib"
scaler_path  = "../models/scaler.joblib"
processed_folder_path  ="../data/processed/"
interim_folder_path = "../data/interim/"

In [5]:
#df_raw = pd.read_csv("../data/raw/beer_reviews.csv")
df_raw = pd.read_csv(rawfile_path)
df_raw.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [6]:
df_raw.info() # 1586614, 13

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_time         1586614 non-null  int64  
 3   review_overall      1586614 non-null  float64
 4   review_aroma        1586614 non-null  float64
 5   review_appearance   1586614 non-null  float64
 6   review_profilename  1586266 non-null  object 
 7   beer_style          1586614 non-null  object 
 8   review_palate       1586614 non-null  float64
 9   review_taste        1586614 non-null  float64
 10  beer_name           1586614 non-null  object 
 11  beer_abv            1518829 non-null  float64
 12  beer_beerid         1586614 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [18]:
# no blank record from target'beer_style'
df_raw[df_raw['beer_style'].isna()]

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid


In [20]:
# read in data_dict to generate a list of features to drop 
#data_dict = pd.read_csv("../references/Data_Dict.csv")
data_dict = pd.read_csv(datadict_path)
features_drop = data_dict.loc[data_dict['API Expected Parameter'] == 'No', 'Column'].tolist()
features_drop.remove('beer_style (target)')
features_drop

['brewery_id',
 'review_time',
 'review_overall',
 'review_profilename',
 'beer_name',
 'beer_beerid']

In [21]:
# drop columns that are not required in API
df_prep = df_raw.drop(features_drop, axis=1)
df_prep.shape

(1586614, 7)

In [22]:
df_prep.info() # 1586614, 7, with beer_style as target

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   brewery_name       1586599 non-null  object 
 1   review_aroma       1586614 non-null  float64
 2   review_appearance  1586614 non-null  float64
 3   beer_style         1586614 non-null  object 
 4   review_palate      1586614 non-null  float64
 5   review_taste       1586614 non-null  float64
 6   beer_abv           1518829 non-null  float64
dtypes: float64(5), object(2)
memory usage: 84.7+ MB


In [24]:
# blank beer_abv
df_prep[df_prep['beer_abv'].isna()]

,brewery_name,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_abv
273,Caldera Brewing Company,3.0,3.0,American Stout,4.0,3.0,NaN
430,Moon River Brewing Company,4.0,4.5,Scotch Ale / Wee Heavy,3.5,3.5,NaN
603,Moon River Brewing Company,3.5,4.0,Scotch Ale / Wee Heavy,3.5,3.5,NaN
733,Caldera Brewing Company,4.0,4.0,American IPA,4.0,4.0,NaN
798,Caldera Brewing Company,4.5,4.0,American Double / Imperial Stout,4.0,4.5,NaN
...,...,...,...,...,...,...,...
1586568,The Defiant Brewing Company,3.5,4.0,Bock,4.0,4.0,NaN
1586587,The Defiant Brewing Company,4.5,4.0,Maibock / Helles Bock,4.5,4.0,NaN
1586596,The Defiant Brewing Company,3.0,5.0,Belgian Strong Pale Ale,4.0,3.5,NaN
1586597,The Defiant Brewing Company,4.5,4.0,Belgian Strong Pale Ale,4.0,4.0,NaN


In [ ]:
# save distinct value of 'beer_style' and 'brewery_name' in case we need it later
#np.save('../data/processed/beer_style', df_prep['beer_style'].unique())
np.save(beer_style_list_path, df_prep['beer_style'].unique())

#np.save('../data/processed/brewery_name', df_prep['brewery_name'].unique())
np.save(brewery_name_list_path, df_prep['brewery_name'].unique())

In [ ]:
# seperate features by type
numeric_features = [col for col in df_prep.columns if col not in ['beer_style','brewery_name']]
## including 'review_aroma','review_appearance','review_palate', 'review_taste' and 'beer_abv', 'beer_style' is target, 'brewery_name' is cate

categorical_features =  [col for col in df_prep.columns if col in ['brewery_name']]
## 'brewery_name' only


In [58]:
# handling blank float column 
# Option A for fill in 'beer_abv' - KNN imputation
from sklearn.impute import KNNImputer

# fro KNN only
n_neighbors=5

df_numeric = df_prep['beer_abv'].copy() # numeric column with NaN to impute
df_categorical = df_prep.drop(columns=['beer_abv']) # save all other columns including numeric and target  df here for KNN purpose only

In [59]:
knn_imputer_numeric = KNNImputer(n_neighbors = n_neighbors)
df_numeric_imputed = knn_imputer_numeric.fit_transform(df_numeric.values.reshape(-1, 1)) ## only fit the 'brewery_abv' column, 67,785 out of 1,586,614
df_imputed = pd.DataFrame(df_numeric_imputed, columns=['beer_abv']) ## convert the imputed column back to df
# runtime = 54m 36.6s

In [62]:
# save the KNN imputer
from joblib import dump
#dump(knn_imputer_numeric, '../models/knn_imputer_numeric.joblib')
dump(knn_imputer_numeric, knn_imputer_path)

['../models/knn_imputer_numeric.joblib']

In [63]:
df_prep_imputed = pd.concat([df_imputed, df_categorical], axis=1)
df_prep_imputed

,beer_abv,brewery_name,review_aroma,review_appearance,beer_style,review_palate,review_taste
0,5.0,Vecchio Birraio,2.0,2.5,Hefeweizen,1.5,1.5
1,6.2,Vecchio Birraio,2.5,3.0,English Strong Ale,3.0,3.0
2,6.5,Vecchio Birraio,2.5,3.0,Foreign / Export Stout,3.0,3.0
3,5.0,Vecchio Birraio,3.0,3.5,German Pilsener,2.5,3.0
4,7.7,Caldera Brewing Company,4.5,4.0,American Double / Imperial IPA,4.0,4.5
...,...,...,...,...,...,...,...
1586609,5.2,The Defiant Brewing Company,4.0,3.5,Pumpkin Ale,4.0,4.0
1586610,5.2,The Defiant Brewing Company,5.0,2.5,Pumpkin Ale,2.0,4.0
1586611,5.2,The Defiant Brewing Company,3.5,3.0,Pumpkin Ale,3.5,4.0
1586612,5.2,The Defiant Brewing Company,4.5,4.5,Pumpkin Ale,4.5,4.5


In [ ]:
# Option B for fill in 'beer_abv'
# df_prep['beer_abv'] = df_prep['beer_abv'].fillna(df_prep['beer_abv'].mean())
# df_prep_imputed = df_prep

In [67]:
#df_prep_imputed.to_pickle(interim_folder_path + 'df_prep_imputed.pkl')
#df_prep_imputed = pd.read_pickle(interim_folder_path + 'df_prep_imputed.pkl')

In [12]:
# save 
df_prep_imputed.to_parquet(interim_folder_path + 'df_prep_imputed.parquet')

In [ ]:
df_prep_imputed= pd.read_pickle(interim_folder_path + 'df_prep_imputed.parquet')

In [13]:
# look in record with NaN in 'brewery_name', 15 out of 1,586,614
df_prep_imputed[df_prep_imputed['brewery_name'].isna()]

,beer_abv,brewery_name,review_aroma,review_appearance,beer_style,review_palate,review_taste
651565,5.0,NaN,2.5,2.5,Vienna Lager,1.5,1.5
659293,7.2,NaN,4.5,3.5,Bock,4.0,3.5
659299,5.4,NaN,3.0,3.0,Dortmunder / Export Lager,4.0,4.0
659300,4.8,NaN,4.0,3.5,Munich Helles Lager,3.5,3.0
659301,5.4,NaN,4.0,4.0,Keller Bier / Zwickel Bier,4.0,3.5
659302,5.9,NaN,4.0,3.0,Vienna Lager,3.0,3.5
659303,5.3,NaN,3.0,3.0,Keller Bier / Zwickel Bier,2.0,3.0
659304,5.3,NaN,4.0,4.0,Keller Bier / Zwickel Bier,4.0,4.5
659305,5.3,NaN,3.5,4.0,Keller Bier / Zwickel Bier,4.0,4.0
1391043,3.8,NaN,3.5,3.5,American Adjunct Lager,4.0,3.5


In [15]:
# Hash encoding column 'brewery_name'
from src.data.sets import apply_hashing_trick

categorical_column = 'brewery_name'

num_buckets = 10

df_prep_imputed = apply_hashing_trick(df_prep_imputed, categorical_column, num_buckets)
df_prep_imputed.drop('brewery_name', axis=1, inplace=True)


In [16]:
# LabelEncoder for target 
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_prep_imputed['beer_style'] = label_encoder.fit_transform(df_prep_imputed['beer_style'])
print(df_prep_imputed.head())


   beer_abv  review_aroma  review_appearance  beer_style  review_palate   
0       5.0           2.0                2.5          65            1.5  \
1       6.2           2.5                3.0          51            3.0   
2       6.5           2.5                3.0          59            3.0   
3       5.0           3.0                3.5          61            2.5   
4       7.7           4.5                4.0           9            4.0   

   review_taste  hashed_brewery_name  
0           1.5                    6  
1           3.0                    6  
2           3.0                    6  
3           3.0                    6  
4           4.5                    5  


In [17]:
# Pop Target 
from src.data.sets import pop_target
df_imputed_prep, target = pop_target(df=df_prep_imputed, target_col='beer_style')

In [18]:
# scale the numeric after KNN imput on NaN
from joblib import load
#scaler = load('../models/scaler.joblib')
scaler = load(scaler_path)

features = pd.DataFrame(scaler.fit_transform(df_imputed_prep), columns=df_imputed_prep.columns)

In [19]:
# Split data set
from src.data.sets import split_sets_random

X_train_v2, y_train_v2, X_val_v2, y_val_v2, X_test_v2, y_test_v2 = split_sets_random(features, target=target, test_ratio=0.2, to_numpy=True)

In [21]:
# save them 
from src.data.sets import save_sets_v2

#save_sets_v2(X_train_v2, y_train_v2, X_val_v2, y_val_v2, X_test_v2, y_test_v2, path = '../data/processed/')
save_sets_v2(X_train_v2, y_train_v2, X_val_v2, y_val_v2, X_test_v2, y_test_v2, path = processed_folder_path, suffix='_v2')